## Model Compilation Jupyter Notebook Example

This notebook shows the example of model compilation using edgeai-benchmark.

This script uses TIDL to compile a model and output in a format that edgeai-sdk can understand.

jai_benchmark is a python package provided in edgeai-benchmark that provides several functions to assist model compilation.

In [1]:
import os
import tempfile
import argparse
import cv2
from jai_benchmark import *

In [2]:
# the cwd must be the root of the respository
if os.path.split(os.getcwd())[-1] in ('scripts', 'tutorials'):
    os.chdir('../')
#
print(os.environ['TIDL_TOOLS_PATH'])
print(os.getcwd())

/data/ssd/files/a0393608/work/code/ti/edgeai-algo/edgeai-benchmark/tidl_tools
/data/ssd/files/a0393608/work/code/ti/edgeai-algo/edgeai-benchmark


#### Create a temporary directory. 

This is were the compiled artifacts will be stored.

In [3]:
modelartifacts_tempdir = tempfile.TemporaryDirectory()
print(modelartifacts_tempdir)
modelartifacts_custom = os.path.join(modelartifacts_tempdir.name, 'modelartifacts')

<TemporaryDirectory '/tmp/tmpxnd97n5t'>


#### Read settings from settings_import_on_pc.yaml

Modify the settings as necessary in the constructor of settings.ConfigSettings()

In [4]:
settings = config_settings.ConfigSettings('./settings_import_on_pc.yaml', 
                modelartifacts_path=modelartifacts_custom,
                calibration_frames=10, calibration_iterations=10, num_frames=100)

work_dir = os.path.join(settings.modelartifacts_path, f'{settings.tensor_bits}bits')
print(f'work_dir = {work_dir}')

work_dir = /tmp/tmpxnd97n5t/modelartifacts/32bits


#### Create Dataset Reader classes

Change the dataset paths according to your dataset location

In [5]:
dataset_calib_cfg = dict(
    path=f'{settings.datasets_path}/coco',
    split='val2017',
    shuffle=True,
    num_frames=min(settings.calibration_frames,5000),
    name='coco'
)

# dataset parameters for actual inference
dataset_val_cfg = dict(
    path=f'{settings.datasets_path}/coco',
    split='val2017',
    shuffle=False, # can be set to True as well, if needed
    num_frames=min(settings.num_frames,5000),
    name='coco'
)

calib_dataset = datasets.COCODetection(**dataset_calib_cfg, download=True)
val_dataset = datasets.COCODetection(**dataset_val_cfg, download=True)


INFO:20220329-161215: dataset exists - will reuse - ./dependencies/datasets/coco
loading annotations into memory...
Done (t=0.71s)
creating index...
index created!

INFO:20220329-161216: dataset exists - will reuse - ./dependencies/datasets/coco
loading annotations into memory...
Done (t=0.92s)
creating index...
index created!


#### Session runtime_options

The default runtime_options can be overriden by passing a runtime_options dict to this function

In [6]:
# choose one session_name depending on the model type
# tflitert for tflite models, onnxrt for onnx models, tvmdlr for mxnet models.
session_name = constants.SESSION_NAME_TFLITERT
#session_name = constants.SESSION_NAME_ONNXRT
#session_name = constants.SESSION_NAME_TVMDLR

session_type = settings.get_session_type(session_name)
runtime_options = settings.get_runtime_options(session_name, is_qat=False)

print(session_type)
print(runtime_options)

<class 'jai_benchmark.sessions.tflitert_session.TFLiteRTSession'>
{'tensor_bits': 32, 'accuracy_level': 1, 'debug_level': 0, 'priority': 0, 'advanced_options:high_resolution_optimization': 0, 'advanced_options:pre_batchnorm_fold': 1, 'advanced_options:calibration_frames': 10, 'advanced_options:calibration_iterations': 10, 'advanced_options:quantization_scale_type': 0, 'advanced_options:activation_clipping': 1, 'advanced_options:weight_clipping': 1, 'advanced_options:bias_calibration': 1, 'advanced_options:channel_wise_quantization': 0, 'advanced_options:output_feature_16bit_names_list': '', 'advanced_options:params_16bit_names_list': '', 'advanced_options:add_data_convert_ops': 3}


In [7]:
preproc_transforms = preprocess.PreProcessTransforms(settings)
postproc_transforms = postprocess.PostProcessTransforms(settings)

# these session cfgs also has some default input mean and scale. 
# if your model needs a difference mean and scale, update the session cfg dict being used with those values
onnx_session_cfg = sessions.get_onnx_session_cfg(settings, work_dir=work_dir)
tflite_session_cfg = sessions.get_tflite_session_cfg(settings, work_dir=work_dir)

#### Create pipeline_configs

pipeline_configs is nothing but a dict with the various model configs that we want to compile

In [8]:
pipeline_configs = {
    'od-mlpefmnv1': dict(
        task_type='detection',
        calibration_dataset=calib_dataset,
        input_dataset=val_dataset,
        preprocess=preproc_transforms.get_transform_tflite((300,300), (300,300), backend='cv2'),
        session=session_type(**tflite_session_cfg,
            runtime_options=runtime_options,
            model_path=f'{settings.models_path}/vision/detection/coco/mlperf/ssd_mobilenet_v1_coco_20180128.tflite'),
        postprocess=postproc_transforms.get_transform_detection_tflite(),
        metric=dict(label_offset_pred=datasets.coco_det_label_offset_90to90()),
        model_info=dict(metric_reference={'accuracy_ap[.5:.95]%':23.0})
    )
}
print(pipeline_configs)

{'od-mlpefmnv1': {'task_type': 'detection', 'calibration_dataset': <jai_benchmark.datasets.coco_det.COCODetection object at 0x7f75da1296a0>, 'input_dataset': <jai_benchmark.datasets.coco_det.COCODetection object at 0x7f75da129470>, 'preprocess': <jai_benchmark.preprocess.PreProcessTransforms object at 0x7f7685ec2b70>, 'session': <jai_benchmark.sessions.tflitert_session.TFLiteRTSession object at 0x7f75cf3401d0>, 'postprocess': <jai_benchmark.postprocess.PostProcessTransforms object at 0x7f75cf3404a8>, 'metric': {'label_offset_pred': {0: 1, 1: 2, 2: 3, 3: 4, 4: 5, 5: 6, 6: 7, 7: 8, 8: 9, 9: 10, 10: 11, 11: 12, 12: 13, 13: 14, 14: 15, 15: 16, 16: 17, 17: 18, 18: 19, 19: 20, 20: 21, 21: 22, 22: 23, 23: 24, 24: 25, 25: 26, 26: 27, 27: 28, 28: 29, 29: 30, 30: 31, 31: 32, 32: 33, 33: 34, 34: 35, 35: 36, 36: 37, 37: 38, 38: 39, 39: 40, 40: 41, 41: 42, 42: 43, 43: 44, 44: 45, 45: 46, 46: 47, 47: 48, 48: 49, 49: 50, 50: 51, 51: 52, 52: 53, 53: 54, 54: 55, 55: 56, 56: 57, 57: 58, 58: 59, 59: 60, 

#### Model Compilation

This will take a few minutes. Please be patient...

In [9]:
# run the model compliation/import and inference
tools.run_accuracy(settings, work_dir, pipeline_configs)

configs to run: ['od-mlpefmnv1_tflitert_coco_mlperf_ssd_mobilenet_v1_coco_20180128_tflite']
number of configs: 1


TASKS                                                       |   0%|          || 0/1 [00:00<?, ?it/s]

TASKS                                                       |          |     0% 0/1| [< ]
INFO:20220329-161254: starting process on parallel_device - 0
INFO:20220329-161259: model_path - /data/ssd/files/a0393608/work/code/ti/edgeai-algo/edgeai-modelzoo/models/vision/detection/coco/mlperf/ssd_mobilenet_v1_coco_20180128.tflite
INFO:20220329-161259: model_file - /tmp/tmpxnd97n5t/modelartifacts/32bits/od-mlpefmnv1_tflitert_coco_mlperf_ssd_mobilenet_v1_coco_20180128_tflite/model/ssd_mobilenet_v1_coco_20180128.tflite
Download done for /data/ssd/files/a0393608/work/code/ti/edgeai-algo/edgeai-modelzoo/models/vision/detection/coco/mlperf/ssd_mobilenet_v1_coco_20180128.tflite
/tmp/tmpxnd97n5t/modelartifacts/32bits/od-mlpefmnv1_tflitert_coco_mlperf_ssd_mobilenet_v1_coco_20180128_tflite/model/ssd_mobilenet_v1_coco_20180128.tflite

INFO:20220329-161300: running - od-mlpefmnv1_tflitert_coco_mlperf_ssd_mobilenet_v1_coco_20180128_tflite
INFO:20220329-161300: pipeline_config - {'task_type': 'detection'

infer : od-mlpefmnv1_tflitert_coco_mlperf_ssd_mobilenet_v1_c|   0%|          || 0/100 [00:00<?, ?it/s]

infer : od-mlpefmnv1_tflitert_coco_mlperf_ssd_mobilenet_v1_c|          |     0% 0/100| [< ]
INFO:20220329-161627: infer completed  - od-mlpefmnv1_tflitert_coco_mlperf_ssd_mobilenet_v1_coco_20180128_tflite - 187 secLoading and preparing results...
DONE (t=0.03s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=1.93s).
Accumulating evaluation results...
DONE (t=0.42s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.329
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.507
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.344
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.039
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.282
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.646
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.286
 Average Recal

#### Package artifacts

Package the artifacts into a .tar.gz file, keeping only the necessary files for inference.

In [10]:
out_dir = f'{work_dir}_package'
tools.package_artifacts(settings, work_dir, out_dir)

packaging artifacts to /tmp/tmpxnd97n5t/modelartifacts/32bits_package please wait...
SUCCESS:20220329-161638: finished packaging - /tmp/tmpxnd97n5t/modelartifacts/32bits/od-mlpefmnv1_tflitert_coco_mlperf_ssd_mobilenet_v1_coco_20180128_tflite


#### Download

Download the packaged .tar.gz artifact

TODO: add a download link here, that the user can click to download the packaged artifact .tar.gz file.

In [11]:
print(f'download the atricats files from the folder: {out_dir}')
print(os.listdir(out_dir))

download the atricats files from the folder: /tmp/tmpxnd97n5t/modelartifacts/32bits_package
['artifacts.yaml', 'artifacts.list', 'od-mlpefmnv1_tflitert_coco_mlperf_ssd_mobilenet_v1_coco_20180128_tflite.tar.gz', 'extract.sh']


#### Cleanup

Remove the temporary folders that we created

In [12]:
# cleanup
modelartifacts_tempdir.cleanup()